2) A agência de aviação de um país monitora voos de duas companhias aéreas para avaliar qual delas é melhor.

Critério: atraso (mais de 15 minutos do previsto). Foram 5 voos por dia, durante 20 dias, por companhia.

Os dados estão na planilha **Green_Belt_FM2S_Lista_Exercícios_05**, aba **02.Cia_Aérea**.

| **Coluna da Worksheet** | **Descrição** |
|--------------------------|---------------|
| **Dia**       | Dia do voo. |
| **Cia**       | Companhia aérea responsável pelo voo. |
| **Atrasado?** | Sim ou Não. |

---

**Responda:**

**a)** No geral, podemos ver diferenças entre a porcentagem de atraso para cada cia?  
Faça o Gráfico de Frequência mais apropriado para os dados.

**b)** Como está o comportamento da porcentagem ao longo do tempo? Existem causas especiais?

**c)** As causas especiais acontecem isoladamente para alguma Cia?

**d)** O que deveríamos esperar de cada uma delas?
